In [5]:
import pandas as pd
import pickle

In [6]:
# read the products with their details
df_products = pd.read_pickle('data/final/product_details/df_product_details_retailrocket.pkl')
# read our recommendations
df_recs = pd.read_pickle('data/final/recommendations/df_recos_category_1-5_new.pkl')

In [7]:
# read the retailrocket sessions
df_recs_retailrocket = pd.read_pickle('data/final/sessions/df_sessions_retailrocket_window_size_1new.pkl')

In [8]:
df_recs.head(1)

,doc_id,term,term_id,tfidf,product_id,Category2,SubCategory2,Minhash_id,Minhash,recs
0,0,"[942411, 177151, 64816, 133116, 1128577, 22797...","[31663, 10456, 23415, 8984, 3499, 11863, 5346,...","[0.12998190993696945, 0.31411559559438207, 0.3...",291285,1426_cat,1589_subcat,m0,<datasketch.weighted_minhash.WeightedMinHash o...,"[m1410, m15433, m15949, m5758, m11035, m9664, ..."


In [9]:
df_recs.head(2)

,doc_id,term,term_id,tfidf,product_id,Category2,SubCategory2,Minhash_id,Minhash,recs
0,0,"[942411, 177151, 64816, 133116, 1128577, 22797...","[31663, 10456, 23415, 8984, 3499, 11863, 5346,...","[0.12998190993696945, 0.31411559559438207, 0.3...",291285,1426_cat,1589_subcat,m0,<datasketch.weighted_minhash.WeightedMinHash o...,"[m1410, m15433, m15949, m5758, m11035, m9664, ..."
1,1,"[1128577, 1249027, 820477, 72034, 912996, 4046...","[3499, 6766, 28250, 25483, 30830, 16813, 14118...","[0.15023119822454753, 0.13586146435157057, 0.2...",103012,561_cat,1483_subcat,m1,<datasketch.weighted_minhash.WeightedMinHash o...,"[m11393, m3882, m12858, m8269, m18731, m12519,..."


In [10]:
df_recs_retailrocket.head(2)

,product1,subcategory_p1,category_p1,product2,subcategory_p2,category_p2,N_products
9,29,1051.0,955.0,"[268335, 29, 442645]","[1051.0, 1051.0, 1051.0]","[955.0, 955.0, 955.0]",3
14,42,84.0,1308.0,"[56782, 336121, 288265, 182676, 148123, 149550...","[1120.0, 84.0, 84.0, 959.0, 84.0, 84.0, 1135.0...","[250.0, 1308.0, 1308.0, 1095.0, 1308.0, 1308.0...",31


In [11]:
df_recs_retailrocket.shape

(14855, 7)

In [12]:
# expand each row to as many rows as the length of the product2 list
df_recs_retailrocket2 = df_recs_retailrocket.set_index('product1').product2.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(int).reset_index()
# rename the columns
df_recs_retailrocket2.columns = ['product_id', 'rec_id']

# convert 'rec_id' to int
#df_recs_retailrocket2['rec_id'] = df_recs_retailrocket2['rec_id'].astype('int')

In [13]:
df_recs_retailrocket2.head(2)

,product_id,rec_id
0,29,268335
1,29,29


In [14]:
df_recs_retailrocket2.shape

(180776, 2)

In [15]:
# keep a matching between 'product_id' & 'Minhash_id'
df_m = df_recs[['product_id', 'Minhash_id']]

# convert 'product_id' to int
#df_m['product_id'] = df_m['product_id'].astype('int')

In [16]:
df_m.head(1)

,product_id,Minhash_id
0,291285,m0


In [17]:
#df_recs_retailrocket2['rec_id'] = df_recs_retailrocket2['rec_id'].astype('str')
#df_m['product_id'] = df_m['product_id'].astype('str')

In [18]:
# add the 'Minhash_id' to each scraped recommendation
df_recs_retailrocket2 = df_recs_retailrocket2.merge(df_m, left_on='rec_id', right_on='product_id', how='left')

In [19]:
df_recs_retailrocket2.head(2)

,product_id_x,rec_id,product_id_y,Minhash_id
0,29,268335,268335.0,m23525
1,29,29,29.0,m24287


In [20]:
df_recs_retailrocket2.product_id_x.nunique()

14855

In [21]:
df_recs_retailrocket2.rec_id.nunique()

30099

In [22]:
df_recs_retailrocket2.Minhash_id.nunique()

14411

In [23]:
# drop recommendations that we dont have details. So, they are from different categories maybe...
df_recs_retailrocket3 = df_recs_retailrocket2.dropna()

In [24]:
df_recs_retailrocket3.head(2)

,product_id_x,rec_id,product_id_y,Minhash_id
0,29,268335,268335.0,m23525
1,29,29,29.0,m24287


In [25]:
df_recs_retailrocket3.shape

(146769, 4)

In [26]:
df_recs_retailrocket3.product_id_x.nunique()

14411

In [27]:
df_recs_retailrocket3.rec_id.nunique()

14411

In [28]:
# group by product and convert recommendations to lists
df_recs_retailrocket4 = df_recs_retailrocket3.groupby(['product_id_x'])['Minhash_id'].apply(list).reset_index(name='retroc_recs')
df_recs_retailrocket4.columns = ['product_id', 'retroc_recs']
df_recs_retailrocket4.head(2)

,product_id,retroc_recs
0,29,"[m23525, m24287, m17866]"
1,42,"[m24222, m2573, m15398, m11685, m21984, m21067..."


In [29]:
df_recs_retailrocket4.shape

(14411, 2)

In [30]:
# keep only the same information of our recommendations
df_recs = df_recs[['product_id', 'recs']]
# convert 'product_id' to int
df_recs['product_id'] = df_recs['product_id'].astype('int')

df_recs.head(2)

,product_id,recs
0,291285,"[m1410, m15433, m15949, m5758, m11035, m9664, ..."
1,103012,"[m11393, m3882, m12858, m8269, m18731, m12519,..."


In [31]:
# create a df with all the recs(scraped & our) 
df_recs_all = df_recs_retailrocket4.merge(df_recs, left_on='product_id', right_on='product_id', how='left')

In [32]:
df_recs_all.head(2)

,product_id,retroc_recs,recs
0,29,"[m23525, m24287, m17866]","[m13992, m21228, m19021, m24287, m26898, m5464..."
1,42,"[m24222, m2573, m15398, m11685, m21984, m21067...","[m2809, m2406, m16289, m11526, m2636, m26916, ..."


In [33]:
len(df_recs_all.iloc[0][2])

10

In [34]:
# keep only the first 5 recommendation that we made
df_recs_all['rec_top5'] = df_recs_all['recs'].apply(lambda x: x[0:10])

In [35]:
# count how many recs are the same
df_recs_all['N_commons'] = df_recs_all.apply(lambda x: len(set(x['retroc_recs']).intersection(set(x['rec_top5']))), axis=1)

In [36]:
df_recs_all['N_recs_retroc'] = df_recs_all['retroc_recs'].apply(lambda x: len(x))

In [37]:
# keep only products with length>=20
#df_recs_all2 = df_recs_all[df_recs_all.N_recs_retroc>=10]

In [38]:
# # save the list of product ids to a pickle file
# eval_recs_productids = df_recs_all[df_recs_all.N_recs_retroc>=10].product_id.unique()
# with open('data/final/recommendations/eval_recs_productids_new.pkl', 'wb') as f:
#     pickle.dump(eval_recs_productids, f)

# keep the same set of products in every scenario & window_size
with open('data/final/recommendations/eval_recs_productids_new.pkl', 'rb') as f:
    eval_recs_productids = pickle.load(f)
# keep a specific set of 3973 products
df_recs_all2 = df_recs_all[df_recs_all.product_id.isin(eval_recs_productids)]

In [39]:
df_recs_all2.head(2)

,product_id,retroc_recs,recs,rec_top5,N_commons,N_recs_retroc
1,42,"[m24222, m2573, m15398, m11685, m21984, m21067...","[m2809, m2406, m16289, m11526, m2636, m26916, ...","[m2809, m2406, m16289, m11526, m2636, m26916, ...",1,20
3,147,"[m16017, m6213, m20676, m18626, m24772, m13099...","[m16017, m27751, m20676, m22609, m21097, m625,...","[m16017, m27751, m20676, m22609, m21097, m625,...",6,37


In [40]:
n1 = df_recs_all2.shape[0]
n1

3973

In [41]:
n2 = df_recs_all2[df_recs_all2.N_commons>0].shape[0]
n2

3804

In [42]:
100*(n2/n1)

95.7462874402215

### --- W_category = 0.5 ... 3.0

In [ ]:
100*(n2/n1) # 3.0

In [638]:
100*(n2/n1) # 2.5

94.78983136169141

In [681]:
100*(n2/n1) # 2.0

94.78983136169141

In [719]:
100*(n2/n1) # 1.5

77.62396174175686

In [522]:
100*(n2/n1) # simple

94.78983136169141

In [560]:
100*(n2/n1) # 0.5

94.71432167128114

### --- W_subcategory = 0.5 ... 3.0

In [408]:
100*(n2/n1) # 3.0

94.68915177447772

In [352]:
100*(n2/n1) # 2.5

94.86534105210168

In [314]:
100*(n2/n1) # 2.0

94.86534105210168

In [276]:
100*(n2/n1) # 1.5

94.84017115529826

In [236]:
100*(n2/n1) # simple

94.78983136169141

In [198]:
100*(n2/n1) # 0.5

94.73949156808456

In [720]:
df_recs_all2.describe()

,product_id,N_commons,N_recs_retroc
count,3973.000000,3973.000000,3973.000000
mean,234823.420337,1.576642,30.995721
std,132748.778598,1.419359,26.397863
min,42.000000,0.000000,10.000000
25%,122350.000000,1.000000,14.000000
50%,236949.000000,1.000000,21.000000
75%,349695.000000,2.000000,38.000000
max,466861.000000,9.000000,245.000000


### -- W_subcategory = 3.0

In [380]:
100*(n2/n1) # window_size = 3 & df_recos_subcategory_3-0.pkl

88.50206138341731

In [304]:
100*(n2/n1) # window_size = 2 & df_recos_subcategory_3-0.pkl

87.31103985341274

In [342]:
100*(n2/n1) # window_size = 1 & df_recos_subcategory_3-0.pkl

84.88318827301879

### -- W_subcategory = 2.5

In [228]:
100*(n2/n1) # window_size = 3 & df_recos_subcategory_2-5.pkl

87.40265689418231

In [190]:
100*(n2/n1) # window_size = 2 & df_recos_subcategory_2-5.pkl

86.16582684379294

In [152]:
100*(n2/n1) # window_size = 1 & df_recos_subcategory_2-5.pkl

83.82959230416857

### -- W_subcategory = 2

In [114]:
100*(n2/n1) # window_size = 3 & df_recos_subcategory_2-0.pkl

87.31103985341274

In [76]:
100*(n2/n1) # window_size = 2 & df_recos_subcategory_2-0.pkl

86.34906092533211

In [38]:
100*(n2/n1) # window_size = 1 & df_recos_subcategory_2-0.pkl

83.737975263399

### -- W_subcategory = 1-5

In [456]:
100*(n2/n1) # window_size = 3 & df_recos_subcategory_1-5.pkl

99.49610627576729

In [532]:
100*(n2/n1) # window_size = 2 & df_recos_subcategory_1-5.pkl

99.31287219422813

In [494]:
100*(n2/n1) # window_size = 1 & df_recos_subcategory_1-5.pkl

98.57993586807147

### -- W_subcategory = 0-5

In [271]:
100*(n2/n1) # window_size = 3 & df_recos_subcategory_0-5.pkl

85.8909757214842

In [233]:
100*(n2/n1) # window_size = 2 & df_recos_subcategory_0-5.pkl

84.88318827301879

In [570]:
100*(n2/n1) # window_size = 1 & df_recos_subcategory_0-5.pkl

98.57993586807147

### -- Simple version

In [157]:
100*(n2/n1) # window_size = 3 & simple tfidf

86.4406779661017

In [119]:
100*(n2/n1) # window_size = 2 & simple tfidf

85.43289051763628

In [39]:
100*(n2/n1) # window_size = 1 & simple tfidf

82.91342189647274